<h1><center> PPOL 5203 : Final Project  <br><br> 
<font color='grey'> Visualizations <br><br>
Ibadat Jarg and Helen Wang</center></center> <h1> 

In [ ]:
#!pip install geopandas

In [ ]:
#setting libraries
import pandas as pd
from plotnine import * 
import geopandas as gpd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
#reading in file
df = pd.read_csv("../data/cleaned_data/final_data.csv")
gdf_points = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['long'], df['lat']), crs="EPSG:4326")

In [ ]:
#dataset of only the opened in 2013
df_13 = df[df["open_year"] == 2013.0]

In [ ]:
#reading in geographic data (zipped on github right now due to size)
america = gpd.read_file('../data/raw_data/county_shape_file_zip')

In [ ]:
#list of counties in DMV
dmv_counties = [
    ('11','001'),  # Washington, D.C.
    ('24', '031'),  # Montgomery County, MD
    ('24', '033'),  # Prince George's County, MD
    ('51', '013'),  # Arlington County, VA
    ('51', '059'),  # Fairfax County, VA
    ('51', '600'),   # Falls Church, VA
    ('51', '510')   # Alexandria, VA
]

In [ ]:
#filtering for DMV counties
dmv = america[
    america.apply(lambda x: (x['STATEFP'], x['COUNTYFP']) in dmv_counties, axis=1)
]

In [ ]:
#plot of bikeshares opened as of today
map1 = (
ggplot(data = df) +
    geom_map(data = dmv,
            mapping = aes(fill = 'NAME', geometry = 'geometry'),
            color = "white",
            alpha = 0.5) +
    geom_point(mapping = aes(x = 'long', y = 'lat'),
               color = "black",
               fill = "#c3d67aff",
               size = 0.5,
               alpha = 0.5
        ) +
    labs(title = "Capital Bike Shares Opened as of 2024", fill = "County") +
    theme_void() +
    theme(
      plot_title=element_text(face="bold")
    ) +
    scale_fill_brewer(type="qual", palette="Set3")
)

map1

In [ ]:
#plot of bikeshares opened in 2013
(
ggplot(data = df_13) +
    geom_map(data = dmv,
            mapping = aes(fill = 'NAME', geometry = 'geometry'),
            color = "white",
            alpha = 0.5) +
    geom_point(mapping = aes(x = 'long', y = 'lat'),
               color = "black",
               fill = "#c3d67aff",
               size = 0.5,
               alpha = 0.5
        ) +
    labs(title = "Capital Bike Shares Opened in 2013", fill = "County") +
    theme_void() +
    theme(
      plot_title=element_text(face="bold")
    ) +
    scale_fill_brewer(type="qual", palette="Set3")
)

In [ ]:
#turned from wide to long
df_13_traffic = df_13.melt(id_vars=['id', 'long', 'lat', 'open_year', 'name', 'opened_station', 'opened_2013'], var_name = "year", value_name = "aadt")

In [ ]:
#treat year as category
df_13_traffic['year'] = df_13_traffic['year'].astype('category')

In [ ]:
#plot of bikeshares opened in 2013 colored by traffic volume 
(
ggplot(data = df_13_traffic[df_13_traffic['year'].isin(["2007", "2019"])]) +
    geom_map(data = dmv,
            mapping = aes(fill = 'NAME', geometry = 'geometry'),
            color = "white",
            alpha = 0.5) +
    geom_point(mapping = aes(x = 'long', y = 'lat', color = "aadt"),
               size = 1,
               alpha = 0.5
        ) +
    labs(title = "Traffic in Capital Bike Share Locations", fill = "County", color = "AADT") +
    theme_void() +
    theme(
      plot_title=element_text(face="bold")
    ) +
    facet_grid('~year') +
    scale_color_distiller(type = "div",palette="RdYlGn", direction = 1) +
    scale_fill_brewer(type="qual", palette="Set3")
)

In [ ]:
#computing change in aadt
df_13["change"] = (df_13["2007"]-df_13["2019"])/df_13["2007"] * 100

In [ ]:
#plot of change in aadt for opened 2013, 2010-2016
(
ggplot(data = df_13) +
    geom_map(data = dmv,
            mapping = aes(fill = 'NAME', geometry = 'geometry'),
            color = "white",
            alpha = 0.5) +
    geom_point(mapping = aes(x = 'long', y = 'lat', color = "change"),
               size = 1,
               alpha = 0.5
        ) +
    labs(title = "Change in Traffic Volume between 2007-2019", fill = "County",
        color = "Change in AADT") +
    theme_void() +
    theme(
      plot_title=element_text(face="bold")
    )+
    scale_fill_brewer(type="qual", palette="Set3") +
    scale_color_distiller(type = "div",palette="RdYlGn", direction = 1,
                        limits=(-50, 50),
                         breaks=[-50, 0, 50])
)

In [ ]:
#computing change in aadt for all stations opened in 2024
df["change_ex"] = (df["2007"]-df["2019"])/df["2007"] * 100

In [ ]:
#plot of change in aadt for all, 2007-2019
(
ggplot(data = df) +
    geom_map(data = dmv,
            mapping = aes(fill = 'NAME', geometry = 'geometry'),
            color = "white",
            alpha = 0.5) +
    geom_point(mapping = aes(x = 'long', y = 'lat', color = "change_ex", shape = "opened_2013"),
               size = 1,
               alpha = 0.5
        ) +
    labs(title = "Change in Traffic Volume between 2007-2019", fill = "County",
        color = "Change in AADT", shape = "Opened in 2013") +
    theme_void() +
    theme(
      plot_title=element_text(face="bold")
    )+
    scale_fill_brewer(type="qual", palette="Set3") +
    scale_color_distiller(type = "div",palette="RdYlGn", direction = 1,
                        limits=(-50, 50),
                         breaks=[-50, 0, 50])
)

In [ ]:
#getting stations that are unopened
proposed = df[df["opened_station"] == 0]

In [ ]:
#plot of proposed, unopened bikeshares 
(
ggplot(data = proposed) +
    geom_map(data = dmv,
            mapping = aes(fill = 'NAME', geometry = 'geometry'),
            color = "white",
            alpha = 0.5) +
    geom_point(mapping = aes(x = 'long', y = 'lat'),
               color = "black",
               fill = "#c3d67aff",
               size = 0.5,
               alpha = 0.5
        ) +
    labs(title = "Proposed Unopened Bikeshare Locations", fill = "County") +
    theme_void() +
    theme(
      plot_title=element_text(face="bold")
    )+
    scale_fill_brewer(type="qual", palette="Set3")
)

In [ ]:
#cleans up .0 after year
df['open_year'] = df['open_year'].replace([np.nan, np.inf, -np.inf], 2000).astype(int).replace(2000, None)

In [ ]:
#number of stations opened per year
stations = df['open_year'].value_counts().reset_index().sort_values(by="open_year", ascending=True)

In [ ]:
#gets total opened stations per year
stations['sum'] = stations['count'].cumsum()

In [ ]:
#compute number of stations with unknown opening years
df['open_year'].isnull().sum()

In [ ]:
stations

In [ ]:
#graph of bikeshares by year
bargraph1 = (
ggplot(data = stations) +
    geom_col(aes(x = "open_year", y = "sum"), color = "red", fill = "white") +
    geom_col(aes(x = "open_year", y = "count"), fill = "coral") +
    labs(x = "Opening Year",
        y = "Total # of Capital Bikeshares",
        title = "Capital Bikeshares by Year",
        caption = "294 stations not included with unknown opening dates",) +
    theme_minimal() +
    scale_x_continuous(breaks = range(2010, 2023))
    
)

bargraph1

In [ ]:
#melting from wide to long
traffic = df.filter(["open_year", '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019']).melt(id_vars= ["open_year"], value_vars =['2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019'])

In [ ]:
#aggregated traffic vol by year
traffic = traffic.groupby("variable")["value"].agg("mean").reset_index()

In [ ]:
#converting it to int for graph
traffic["variable"] = traffic["variable"].astype(int)

In [ ]:
#linegraph of traffic vol by year
linegraph1 = (
ggplot(data = traffic) +
    geom_point(aes(x = "variable", y = "value"), color = "black", size = 1.5) +
    geom_line(aes(x = "variable", y = "value", group = 1), color = "red", size = 1.5) +
    scale_y_continuous(limits = [0, 30000]) +
    scale_x_continuous(limits = [2007, 2019], breaks = range(2007, 2019)) +
    labs(x = "Year",  y = "Average AADT", title = "Traffic Volume per Year") +
    theme_minimal() 
)

linegraph1

In [ ]:
df.query("opened_station == True").shape

In [ ]:
traffic_na = df.query("opened_station == True").filter(['2007', '2008', '2009','2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018','2019']).isna().sum().reset_index().rename(columns = {"index": "Year", 0: "Count"}).assign(Percent = lambda x: x["Count"]/780 * 100)

In [ ]:
traffic_na

In [ ]:
linegraph2 = (
ggplot(data = traffic_na) +
    geom_point(aes(x = "Year", y = "Percent"), color = "black", size = 1.5) +
    geom_line(aes(x = "Year", y = "Percent", group = 1), color = "red", size = 1.5) +
    scale_y_continuous(limits = [0, 100]) +
    labs(x = "Year",  y = "% Missing Data", title = "% of Missing Traffic Volume Data per Year") +
    theme_minimal() 
    
)

linegraph2

Saving graphs

In [ ]:
#saving graphs
map1.save('../visualizations/graphs/map1.png', dpi=300)

In [ ]:
bargraph1.save('../visualizations/graphs/bargraph1.png', dpi=300)

In [ ]:
linegraph1.save('../visualizations/graphs/linegraph1.png', dpi=300)

In [ ]:
linegraph2.save('../visualizations/graphs/bargraph2.png', dpi=300)